## Running Classification Modelling on the created dataset using SparkML in GCP DataProc

Similar to practices, here we will run code to conduct a classification model.

In [1]:
# Import libraries

from __future__ import print_function
from pyspark.context import SparkContext
from pyspark.ml.linalg import Vectors
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer

In [2]:
# Collect features of interest for the model

def vector_from_inputs(r):
    return(r["transactions"], Vectors.dense(float(r["countryIndex"]),
                                           float(r["isMobile"]),
                                           float(r["pageviews"]),
                                           float(r["operatinSystemIndex"])))

In [3]:
spark = SparkSession.builder.config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/08 18:41:28 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/08 18:41:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/08 18:41:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/08 18:41:28 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


## Get BigQuery Data

Since dataset is public we should be able to directly grab it

In [4]:
ga_data = spark.read.format("bigquery").option(
    "table", "bigquery-public-data.google_analytics_sample.ga_sessions_20170801").load()
# Create a view so that Spark SQL queries can be run against the data.
ga_data.createOrReplaceTempView("googleAnalytics")

## -------------------ENSURE CLEAN DATA NO NULLS----------------

sql_query = """
SELECT device.operatingSystem, device.isMobile, geoNetwork.country, totals.pageviews, totals.transactions
from googleAnalytics
where device.operatingSystem is not null
and device.isMobile is not null
and geoNetwork.country is not null
and totals.pageviews is not null
and totals.transactions is not null"""
ga_clean = spark.sql(sql_query)

22/04/08 18:41:38 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
# Show some results to ensure we got the table

ga_clean.show(10)

+---------------+--------+-------------+---------+------------+
|operatingSystem|isMobile|      country|pageviews|transactions|
+---------------+--------+-------------+---------+------------+
|      Macintosh|   false|United States|        5|           1|
|      Macintosh|   false|United States|       11|           1|
|      Macintosh|   false|United States|       14|           1|
|      Macintosh|   false|United States|       14|           1|
|      Macintosh|   false|United States|       16|           1|
|      Macintosh|   false|United States|       14|           1|
|      Macintosh|   false|United States|       13|           1|
|      Macintosh|   false|United States|       16|           1|
|            iOS|    true|      Finland|       18|           1|
|      Macintosh|   false|United States|       18|           1|
+---------------+--------+-------------+---------+------------+
only showing top 10 rows



## Classification Model Prep

Below we need to ensure data is ready to go for the model. We will: 

    - conduct indexing/one-hot encoding on categorical variables

In [6]:
#First index columns
data_index = StringIndexer(inputCols=['operatingSystem', 'country'], 
                           outputCols=['operatinSystemIndex', 'countryIndex'])

data_indexed = data_index.fit(ga_clean).transform(ga_clean)


"\n# Once indexed, we can now use a one-hot approach on the data. \ndata_one_hot = OneHotEncoder(inputCols=['operatinSystemIndex', 'countryIndex'],\n                       outputCols=['operatingSystemEncoded', 'countryEncoded'])\n\ndata_encoded = data_one_hot.fit(data_indexed).transform(data_indexed)\n"

In [7]:
#View
data_indexed.show(10)

+---------------+--------+-------------+---------+------------+-------------------+------------+
|operatingSystem|isMobile|      country|pageviews|transactions|operatinSystemIndex|countryIndex|
+---------------+--------+-------------+---------+------------+-------------------+------------+
|      Macintosh|   false|United States|        5|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       11|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       14|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       14|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       16|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       14|           1|                0.0|         0.0|
|      Macintosh|   false|United States|       13|           1|                0.0|         0.0|
|      Macintosh|   false|Unit

In [8]:
# Next, combine the columns into a useable dataset

model_data = data_indexed.drop("operatingSystem","country")



In [9]:
# Finally, move to training data

training_data = model_data.rdd.map(vector_from_inputs).toDF(["label",
                                                             "features"])
training_data.cache()

DataFrame[label: bigint, features: vector]

## On to the classification model

Personally I have enjoyed using Random Forests in classes so I have finally decided to run with that below.

In [12]:
rfc = RandomForestClassifier(labelCol = 'label', featuresCol = 'features',
                            subsamplingRate = 0.7)

In [13]:
rfc_model = rfc.fit(training_data)

In [14]:
rfc_model.summary

In [16]:
rfc_model.summary.accuracy

0.9534883720930233